In [1]:
# ✅ Necessary Imports
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, random_split
from PIL import Image
import timm
from torch.optim.lr_scheduler import CosineAnnealingLR, LambdaLR

# ✅ Step 1: Setup and Imports
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ✅ Step 2: Dataset Preparation
class CustomDataset(Dataset):
    def __init__(self, root_dir, label, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_files = [(os.path.join(root_dir, f), label) for f in os.listdir(root_dir)]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path, label = self.image_files[idx]
        try:
            image = Image.open(img_path).convert('RGB')
            if self.transform:
                image = self.transform(image)
            return image, label
        except (OSError, IOError) as e:
            print(f"Warning: Skipping corrupted image {img_path}. Error: {e}")
            return self.__getitem__((idx + 1) % len(self))  # Return next valid image

# ✅ Step 3: Data Augmentation
train_transform = transforms.Compose([
    transforms.Resize((300, 300)),  # EfficientNet-B3 uses 300x300 images
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((300, 300)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# ✅ Step 4: Load Datasets
real_dataset = CustomDataset(root_dir='/kaggle/input/vista-25/dataset/dataset/train/real', label=1, transform=train_transform)
fake_dataset = CustomDataset(root_dir='/kaggle/input/vista-25/dataset/dataset/train/fake', label=0, transform=train_transform)

# ✅ Merge and Split for Training and Validation
full_dataset = real_dataset + fake_dataset
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# ✅ DataLoaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=4, pin_memory=True)

# ✅ Step 5: Load EfficientNet-B3 Model
model = timm.create_model("efficientnet_b3", pretrained=False, num_classes=2)  # Adjust for binary classification
model = model.to(device)

# ✅ Step 6: Loss Function (Label Smoothing & Focal Loss)
class FocalLoss(nn.Module):
    """ Focal Loss for class imbalance (α=0.25, γ=2.0 for strong focus) """
    def __init__(self, alpha=0.25, gamma=2.0, reduction='mean', label_smoothing=0.1):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction
        self.label_smoothing = label_smoothing

    def forward(self, inputs, targets):
        ce_loss = nn.CrossEntropyLoss(label_smoothing=self.label_smoothing, reduction='none')(inputs, targets)
        pt = torch.exp(-ce_loss)
        focal_loss = self.alpha * (1 - pt) ** self.gamma * ce_loss

        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss

criterion = FocalLoss()

# ✅ Step 7: Optimizer & Learning Rate Warmup + Cosine Annealing
optimizer = optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-4)

# Learning Rate Warmup Scheduler
def warmup_lambda(epoch):
    warmup_epochs = 3
    if epoch < warmup_epochs:
        return (epoch + 1) / warmup_epochs  # Gradually increase LR
    return 1  # Stay constant after warmup

warmup_scheduler = LambdaLR(optimizer, lr_lambda=warmup_lambda)

# Cosine Annealing LR Scheduler
cosine_scheduler = CosineAnnealingLR(optimizer, T_max=10)

# ✅ Step 8: Training Loop
def train_model(model, train_loader, val_loader, criterion, optimizer, warmup_scheduler, cosine_scheduler, num_epochs=8):
    model_save_path = "/kaggle/working/efficientnet_b3_deepfake.pth"
    try:
        for epoch in range(num_epochs):
            model.train()
            running_loss, correct, total = 0.0, 0, 0

            for images, labels in train_loader:
                images, labels = images.to(device), labels.to(device)

                optimizer.zero_grad()
                outputs = model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                running_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

            train_loss = running_loss / len(train_loader)
            train_acc = correct / total

            # ✅ Validation Phase
            model.eval()
            val_running_loss, val_correct, val_total = 0.0, 0, 0
            with torch.no_grad():
                for images, labels in val_loader:
                    images, labels = images.to(device), labels.to(device)
                    outputs = model(images)
                    loss = criterion(outputs, labels)

                    val_running_loss += loss.item()
                    _, predicted = torch.max(outputs, 1)
                    val_total += labels.size(0)
                    val_correct += (predicted == labels).sum().item()

            val_loss = val_running_loss / len(val_loader)
            val_acc = val_correct / val_total

            # ✅ Apply Learning Rate Schedulers
            if epoch < 3:
                warmup_scheduler.step()  # Apply Warmup LR for first 3 epochs
            else:
                cosine_scheduler.step()  # Apply Cosine Annealing after warmup

            print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, "
                  f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

            # Save checkpoints every 2 epochs
            if (epoch + 1) % 2 == 0:
                checkpoint_path = f"/kaggle/working/efficientnet_epoch_{epoch+1}.pth"
                torch.save(model.state_dict(), checkpoint_path)
                print(f"Checkpoint saved: {checkpoint_path}")

        # Save the final model
        torch.save(model.state_dict(), model_save_path)
        print(f"Model saved successfully to {model_save_path}")
    except Exception as e:
        print(f"Error encountered: {e}. Saving the model to {model_save_path}")
        torch.save(model.state_dict(), model_save_path)
        print(f"Model saved after error.")
        sys.exit(0)

# ✅ Train the model
train_model(model, train_loader, val_loader, criterion, optimizer, warmup_scheduler, cosine_scheduler, num_epochs=8)

# ✅ Save Final Model
from IPython.display import FileLink
FileLink('/kaggle/working/efficientnet_b3_deepfake.pth')

Using device: cuda


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3406: DecompressionBombWarning: Image size (98058240 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3406: DecompressionBombWarning: Image size (96000000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:1054: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:1054: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:1054: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3406: Dec

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:1054: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:1054: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3406: DecompressionBombWarning: Image size (161087488 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:1054: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3406: DecompressionBombWarning: Image size (107184040 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3406: DecompressionBombWarning: Image size (99991727 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


...

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3406: DecompressionBombWarning: Image size (161087488 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:1054: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3406: DecompressionBombWarning: Image size (107184040 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3406: DecompressionBombWarning: Image size (99991727 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3406: DecompressionBombWarning: Image size (96012000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Epoch 8/8, Train Loss: 0.0316, Train Acc: 0.7763, Val Loss: 0.0297, Val Acc: 0.7979
Checkpoint saved: /kaggle/working/efficientnet_epoch_8.pth
Model saved successfully to /kaggle/working/efficientnet_b3_deepfake.pth


/kaggle/working/efficientnet_b3_deepfake.pth